In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n_rows = 10000000

train = pd.read_csv(
    '../input/train.csv.zip', 
    index_col='key', 
    compression='infer', 
    nrows=n_rows)

valid = pd.read_csv(
    '../input/train.csv.zip', 
    index_col='key', 
    compression='infer', 
    nrows=n_rows/10, 
    skiprows=list(range(1,n_rows+1)))

infer = pd.read_csv('../input/test.csv', index_col='key')

In [3]:
def feature_engineering(df):
    df['pickup'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    df['weekday'] = df['pickup'].dt.dayofweek
    df['hour'] = df['pickup'].dt.hour
        
    df['lat_diff'] = abs(df['pickup_latitude'] - df['dropoff_latitude'])
    df['lon_diff'] = abs(df['pickup_longitude'] - df['dropoff_longitude'])
    df['euclidean'] = np.sqrt(df['lat_diff']**2 + df['lon_diff']**2)
        
    df.drop(['pickup_datetime','pickup'], axis=1, inplace=True)

feature_engineering(train)
feature_engineering(valid)
feature_engineering(infer)

train.dropna(how='any', axis=0, inplace=True)
valid.dropna(how='any', axis=0, inplace=True)

In [4]:
def norm_params(df):
    params = {}
    columns = [
        'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 
        'lat_diff', 'lon_diff', 
        'euclidean']
    
    for column in columns:
        #params[column] = {'mean': df[column].mean(), 'std': df[column].std()}
        params[column] = {'min': df[column].min(), 'max': df[column].max()}
        
    return params

params = norm_params(train)

def normalize(df, params):
    for key in params:
        if key == 'euclidean':
            df[key+'_norm'] = np.sqrt(df['lat_diff_norm']**2 + df['lon_diff_norm']**2)
        else:
            #mean = params[key]['mean']
            #std = params[key]['std']
            #df[key+'_norm'] = (df[key] - mean)/std
            
            minimum = params[key]['min']
            maximum = params[key]['max']

            df[key+'_norm'] = (df[key] - minimum) / (maximum - minimum)
            
        
normalize(train, params)
normalize(valid, params)
normalize(infer, params)

In [5]:
y_train = train['fare_amount']
y_valid = valid['fare_amount']

X_train = train.drop('fare_amount', axis=1, inplace=True)
X_valid = valid.drop('fare_amount', axis=1, inplace=True)

In [6]:
# definition of the numeric columns
p_lat = tf.feature_column.numeric_column('pickup_latitude')
p_lon = tf.feature_column.numeric_column('pickup_longitude')
d_lat = tf.feature_column.numeric_column('dropoff_latitude')
d_lon = tf.feature_column.numeric_column('dropoff_longitude')
p_cnt = tf.feature_column.numeric_column('passenger_count')
lat_diff = tf.feature_column.numeric_column('lat_diff')
lon_diff = tf.feature_column.numeric_column('lon_diff')
euclidean = tf.feature_column.numeric_column('euclidean')

# definition of the normalized numeric columns
p_lat_norm = tf.feature_column.numeric_column('pickup_latitude_norm')
p_lon_norm = tf.feature_column.numeric_column('pickup_longitude_norm')
d_lat_norm = tf.feature_column.numeric_column('dropoff_latitude_norm')
d_lon_norm = tf.feature_column.numeric_column('dropoff_longitude_norm')
lat_diff_norm = tf.feature_column.numeric_column('lat_diff_norm')
lon_diff_norm = tf.feature_column.numeric_column('lon_diff_norm')
euclidean_norm = tf.feature_column.numeric_column('euclidean_norm')

# definition of the categorical columns
weekday = tf.feature_column.categorical_column_with_identity('weekday', num_buckets = 7)
hour = tf.feature_column.categorical_column_with_identity('hour', num_buckets = 24)

# definition of the bucketized columns
numbuckets = 16

latbuckets = np.linspace(38,42,numbuckets).tolist()
lonbuckets = np.linspace(-76,-72,numbuckets).tolist()

p_latB = tf.feature_column.bucketized_column(p_lat, latbuckets)
d_latB = tf.feature_column.bucketized_column(d_lat, latbuckets)
p_lonB = tf.feature_column.bucketized_column(p_lon, lonbuckets)
d_lonB = tf.feature_column.bucketized_column(d_lon, lonbuckets)

# definition of the feature crosses
ploc = tf.feature_column.crossed_column([p_latB, p_lonB], numbuckets * numbuckets)
dloc = tf.feature_column.crossed_column([d_latB, d_lonB], numbuckets * numbuckets)
pd_pair = tf.feature_column.crossed_column([ploc, dloc], numbuckets**4)
day_hr = tf.feature_column.crossed_column([weekday, hour], 7 * 24)

In [7]:
wide_columns = [
    # Feature crosses
    ploc, dloc, pd_pair, day_hr, 

    # Sparse columns
    weekday, hour,  

    # Anything with a linear relationship
    p_cnt 
]

In [8]:
deep_columns = [
    # Embedding_column to "group" together ...
    tf.feature_column.embedding_column(pd_pair, 10),
    tf.feature_column.embedding_column(day_hr, 10),

    # Numeric columns
    # p_lat, p_lon, d_lat, d_lon, lat_diff, lon_diff, euclidean
    
    # Normalized numeric columns
    p_lat_norm, p_lon_norm, d_lat_norm, d_lon_norm, lat_diff_norm, lon_diff_norm, euclidean_norm
]

In [9]:
def train_input_fn(features, labels, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(features, labels):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        y = labels,
        num_epochs = 1,
        shuffle = True)

def pred_input_fn(features):
    return tf.estimator.inputs.pandas_input_fn(
        x = features,
        num_epochs = 1,
        shuffle = False)

In [10]:
model_dir = './Google_Example'

file_writer = tf.summary.FileWriter(model_dir)

estimator = tf.estimator.DNNLinearCombinedRegressor(
    model_dir = model_dir,
    linear_feature_columns = wide_columns,
    dnn_feature_columns = deep_columns,
    dnn_hidden_units = [64,64,64,8]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './Google_Example', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000213F0A83828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(train, y_train, batch_size=5000))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(valid, y_valid))

In [13]:
epochs = 89

for i in range(epochs):
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Example\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into ./Google_Example\model.ckpt.
INFO:tensorflow:loss = 234219.84, step = 20000
INFO:tensorflow:global_step/sec: 9.74312
INFO:tensorflow:loss = 254032.66, step = 20100 (10.265 sec)
INFO:tensorflow:global_step/sec: 10.3769
INFO:tensorflow:loss = 215445.72, step = 20200 (9.637 sec)
INFO:tensorflow:global_step/sec: 10.4219
INFO:tensorflow:loss = 26

INFO:tensorflow:loss = 277380.34, step = 24000
INFO:tensorflow:global_step/sec: 9.87639
INFO:tensorflow:loss = 258516.19, step = 24100 (10.128 sec)
INFO:tensorflow:global_step/sec: 10.8467
INFO:tensorflow:loss = 243401.86, step = 24200 (9.217 sec)
INFO:tensorflow:global_step/sec: 11.2468
INFO:tensorflow:loss = 225523.4, step = 24300 (8.891 sec)
INFO:tensorflow:global_step/sec: 11.1677
INFO:tensorflow:loss = 212141.77, step = 24400 (8.954 sec)
INFO:tensorflow:global_step/sec: 11.3168
INFO:tensorflow:loss = 205695.53, step = 24500 (8.836 sec)
INFO:tensorflow:global_step/sec: 11.3383
INFO:tensorflow:loss = 249288.39, step = 24600 (8.820 sec)
INFO:tensorflow:global_step/sec: 10.6891
INFO:tensorflow:loss = 252207.94, step = 24700 (9.355 sec)
INFO:tensorflow:global_step/sec: 11.2879
INFO:tensorflow:loss = 295668.3, step = 24800 (8.859 sec)
INFO:tensorflow:global_step/sec: 11.3652
INFO:tensorflow:loss = 252260.67, step = 24900 (8.799 sec)
INFO:tensorflow:global_step/sec: 10.8596
INFO:tensorfl

INFO:tensorflow:global_step/sec: 11.1473
INFO:tensorflow:loss = 216119.03, step = 28700 (8.956 sec)
INFO:tensorflow:global_step/sec: 11.0276
INFO:tensorflow:loss = 232440.47, step = 28800 (9.069 sec)
INFO:tensorflow:global_step/sec: 11.2046
INFO:tensorflow:loss = 296715.16, step = 28900 (8.923 sec)
INFO:tensorflow:global_step/sec: 11.0137
INFO:tensorflow:loss = 266875.0, step = 29000 (9.095 sec)
INFO:tensorflow:global_step/sec: 11.4784
INFO:tensorflow:loss = 217854.0, step = 29100 (8.696 sec)
INFO:tensorflow:global_step/sec: 11.2007
INFO:tensorflow:loss = 270099.25, step = 29200 (8.928 sec)
INFO:tensorflow:global_step/sec: 11.7418
INFO:tensorflow:loss = 229761.55, step = 29300 (8.517 sec)
INFO:tensorflow:global_step/sec: 11.6553
INFO:tensorflow:loss = 225711.08, step = 29400 (8.580 sec)
INFO:tensorflow:global_step/sec: 11.6553
INFO:tensorflow:loss = 302646.44, step = 29500 (8.582 sec)
INFO:tensorflow:global_step/sec: 11.5656
INFO:tensorflow:loss = 245019.75, step = 29600 (8.644 sec)
IN

INFO:tensorflow:global_step/sec: 11.9015
INFO:tensorflow:loss = 232452.44, step = 33400 (8.418 sec)
INFO:tensorflow:global_step/sec: 11.6691
INFO:tensorflow:loss = 242475.53, step = 33500 (8.554 sec)
INFO:tensorflow:global_step/sec: 11.7692
INFO:tensorflow:loss = 262075.1, step = 33600 (8.497 sec)
INFO:tensorflow:global_step/sec: 11.8745
INFO:tensorflow:loss = 244263.23, step = 33700 (8.421 sec)
INFO:tensorflow:global_step/sec: 11.5735
INFO:tensorflow:loss = 224667.56, step = 33800 (8.640 sec)
INFO:tensorflow:global_step/sec: 11.8744
INFO:tensorflow:loss = 255530.72, step = 33900 (8.437 sec)
INFO:tensorflow:Saving checkpoints for 34000 into ./Google_Example\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-19:20:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Example\model.ckpt-34000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_ini

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-19:26:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Example\model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-08-14-19:26:13
INFO:tensorflow:Saving dict for global step 38000: average_loss = 47.198196, global_step = 38000, label/mean = 11.230434, loss = 6041.369, prediction/mean = 11.329334
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 38000: ./Google_Exam

...

INFO:tensorflow:Saving checkpoints for 72000 into ./Google_Example\model.ckpt.
INFO:tensorflow:loss = 244032.97, step = 72000
INFO:tensorflow:global_step/sec: 9.95603
INFO:tensorflow:loss = 262674.22, step = 72100 (10.045 sec)
INFO:tensorflow:global_step/sec: 10.8336
INFO:tensorflow:loss = 229534.5, step = 72200 (9.230 sec)
INFO:tensorflow:global_step/sec: 10.8158
INFO:tensorflow:loss = 246533.12, step = 72300 (9.247 sec)
INFO:tensorflow:global_step/sec: 10.8639
INFO:tensorflow:loss = 235040.52, step = 72400 (9.205 sec)
INFO:tensorflow:global_step/sec: 10.4642
INFO:tensorflow:loss = 227836.31, step = 72500 (9.556 sec)
INFO:tensorflow:global_step/sec: 10.736
INFO:tensorflow:loss = 274111.7, step = 72600 (9.314 sec)
INFO:tensorflow:global_step/sec: 10.7474
INFO:tensorflow:loss = 242656.06, step = 72700 (9.305 sec)
INFO:tensorflow:global_step/sec: 11.0868
INFO:tensorflow:loss = 238969.03, step = 72800 (9.019 sec)
INFO:tensorflow:global_step/sec: 11.2974
INFO:tensorflow:loss = 223836.9, st

INFO:tensorflow:loss = 233598.16, step = 76600 (9.047 sec)
INFO:tensorflow:global_step/sec: 11.2364
INFO:tensorflow:loss = 216195.19, step = 76700 (8.900 sec)
INFO:tensorflow:global_step/sec: 10.8124
INFO:tensorflow:loss = 285896.34, step = 76800 (9.248 sec)
INFO:tensorflow:global_step/sec: 10.9815
INFO:tensorflow:loss = 231074.34, step = 76900 (9.107 sec)
INFO:tensorflow:global_step/sec: 10.7092
INFO:tensorflow:loss = 248565.83, step = 77000 (9.337 sec)
INFO:tensorflow:global_step/sec: 10.6558
INFO:tensorflow:loss = 283492.5, step = 77100 (9.386 sec)
INFO:tensorflow:global_step/sec: 10.7029
INFO:tensorflow:loss = 233943.56, step = 77200 (9.342 sec)
INFO:tensorflow:global_step/sec: 10.6396
INFO:tensorflow:loss = 235986.73, step = 77300 (9.407 sec)
INFO:tensorflow:global_step/sec: 10.7438
INFO:tensorflow:loss = 319918.38, step = 77400 (9.300 sec)
INFO:tensorflow:global_step/sec: 11.3454
INFO:tensorflow:loss = 262425.2, step = 77500 (8.814 sec)
INFO:tensorflow:global_step/sec: 11.2991
IN

INFO:tensorflow:global_step/sec: 10.5687
INFO:tensorflow:loss = 249425.72, step = 81300 (9.475 sec)
INFO:tensorflow:global_step/sec: 10.3422
INFO:tensorflow:loss = 231911.28, step = 81400 (9.658 sec)
INFO:tensorflow:global_step/sec: 10.9466
INFO:tensorflow:loss = 219539.44, step = 81500 (9.135 sec)
INFO:tensorflow:global_step/sec: 11.6418
INFO:tensorflow:loss = 237692.64, step = 81600 (8.588 sec)
INFO:tensorflow:global_step/sec: 11.2096
INFO:tensorflow:loss = 203978.84, step = 81700 (8.921 sec)
INFO:tensorflow:global_step/sec: 10.9815
INFO:tensorflow:loss = 262975.62, step = 81800 (9.108 sec)
INFO:tensorflow:global_step/sec: 10.875
INFO:tensorflow:loss = 240888.78, step = 81900 (9.195 sec)
INFO:tensorflow:Saving checkpoints for 82000 into ./Google_Example\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-20:35:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Ex

INFO:tensorflow:Saving checkpoints for 86000 into ./Google_Example\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-14-20:41:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Example\model.ckpt-86000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-08-14-20:41:17
INFO:tensorflow:Saving dict for global step 86000: average_loss = 50.524223, global_step = 86000, label/mean = 11.210979, loss = 6467.1006, prediction/mean = 11.282732
INFO:

KeyboardInterrupt: 

In [14]:
generator = estimator.predict(input_fn=pred_input_fn(infer))
predictions = [next(generator) for i in range(len(infer))]
values = [val['predictions'].tolist()[0] for val in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Google_Example\model.ckpt-88000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [15]:
ids = infer.index

submission = pd.DataFrame()
submission['fare_amount'] = values
submission['key'] = ids
submission.set_index('key', inplace=True)

submission.to_csv('../output/04.google_example.csv')